In [4]:
import pyspark

In [5]:
spark = pyspark.sql.SparkSession.builder.appName("DeltaTest") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [6]:
##Batch
data = spark.range(0, 5)
data.write.format("delta").save(r"C:\Users\kndoa\OneDrive\Desktop\Data_Science\CovidDeltaLake\test")

In [7]:
df = spark.read.format("delta").load(r"C:\Users\kndoa\OneDrive\Desktop\Data_Science\CovidDeltaLake\test")
df.show()

+---+
| id|
+---+
|  0|
|  3|
|  1|
|  2|
|  4|
+---+



In [8]:
##Stream
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

In [9]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()